# Equal-Weight S&P 500 Index Fund

In [22]:
import numpy as np 
import pandas as pd
import requests 
import xlsxwriter 
import math 

New libraries:

- requests: used for HTTP requests.

- xlswriter : used to save Excel files from a Python script

### Importing List of Stocks

S&P 500 Index actually changes with time, but in this notebook we will consider a static version of it, which you can find in the folder under the name _sp_500_stocks.csv_. As you wil see, it is a file with the tickers/symbols of each company in S&P 500.



In [23]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [24]:
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## API Token

In this project, we'll be working with the IEX Cloud API token, which will be our data provider. API Tokens should be stored in such a way that they don't get pushed to the Git repository, as it contains sensitive information. This file is the one called `secrets.py`

The API we'll be using generates random data which mimics real-world data. The main reason why we are using this API is because it is free.

In [25]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Making calls in APIs is tricky, as the method is different for each of them. In this case, we would like to extract the following information:

* Market capitalization for each stock
* Price of each stock

This is done by using the code below. Documentation related to the API can be found [here ](https://iexcloud.io/docs/api/) (look for _Testing Sandbox_ and _Quote_):

In [26]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}' #token needed at
#the end to verify that we can have access to that information
data = requests.get(api_url).json() 
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'MAARC NLEESL()OQGBTKN TAAGDSLE/S ',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'folfciai',
 'close': None,
 'closeTime': None,
 'closeSource': 'foaicilf',
 'high': None,
 'highTime': None,
 'highSource': None,
 'low': None,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 126.247,
 'latestSource': 'IEX real time price',
 'latestTime': '3:12:55 PM',
 'latestUpdate': 1645965519052,
 'latestVolume': None,
 'iexRealtimePrice': 123.481,
 'iexRealtimeSize': 61,
 'iexLastUpdated': 1631274089306,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 123.15,
 'previousVolume': 120447141,
 'change': 1.115,
 'changePercent': 0.00933,
 'volume': None,
 'iexMarketPercent': 0.010876665181136736,
 'iexVolume': 6

As you can see, this provides lots of information, but we only want the market capitalization and the latest price of the stock:

In [27]:
price = data["latestPrice"]
market_cap = data["marketCap"]


## Adding Stocks Data to a DataFrame

The title is self-explanatory:

In [28]:
my_columns = ["Ticker","Stock Price",
              "Market Capitalization","Number of Shares to Buy"]

final_dataframe = pd.DataFrame(columns=my_columns)

final_dataframe = final_dataframe.append(pd.Series([symbol,price,market_cap,'N/A'], index = my_columns),
                       ignore_index=True) #in order to append it must be given as pd.Series

final_dataframe



,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,126.247,2104968729519,N/A


## Looping Through Every Ticker in the Stock List

The code below works, but it is _very_ slow, as HTTP requests are extremely slow. You can try the code with the first 10 tickers. Fortunately, there are some methods that improve the performance of the calls:

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"][:10]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}' #token needed at
    data = requests.get(api_url).json() 
    price = data["latestPrice"]
    market_cap = data["marketCap"]
    final_dataframe = final_dataframe.append(pd.Series([stock,price,market_cap,'N/A'], index = my_columns),
                       ignore_index=True) 

        

In [ ]:
final_dataframe.head()

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code. IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [ ]:
def groups(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n] #yield returns a generator

In [ ]:
symbol_groups = list(groups(stocks["Ticker"],100))
symbol_strings = []

for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i])) #we are joining each list and separating them by a comma, basically
    
symbol_strings
    

And now we perform the batch (look for _Batch Requests_ in API documentation):

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
     batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
     data = requests.get(batch_api_call_url).json()
     for symbol in symbol_string.split(','): #opposite of join:
        final_dataframe = final_dataframe.append(
            pd.Series(
             [symbol,
              data[symbol]['quote']['latestPrice'], #for some reason it is inside something called quote
              data[symbol]['quote']['marketCap'],
               "N/A"],
            index=my_columns),
            ignore_index=True)
        

In [ ]:
final_dataframe.head()

Since we are making 5 calls instead of 500 (we have 5 groups of 100 tickers), this piece of code is _much_ faster.

## Calculating the Number of Shares To Buy

First of all, we enter with the keyboard the value of our portfolio:

In [ ]:
portfolio_size = input("Enter the value of your portfolio")

try:
    val = float(portfolio_size)
except ValueError:
    print("Please, enter a numerical value")
    portfolio_size = input("Enter the value of  your portfolio")
    val=float(portfolio_size)

And now we calculate the size of the positions for each ticker. Once we have done this, we can update the row of the DataFrame corresponding to the number of shares to buy, as this is just the price of the stocks divided by its size in our portfolio:

In [ ]:
position_size = val/len(final_dataframe.index) #since we want EW portfolio, the size of each position is the same

final_dataframe["Number of Shares to Buy"] = position_size/final_dataframe["Stock Price"]



**BEAR IN MID THAT WE HAVE FRACTIONAL SHARES**

## Formatting An Excel Output

For this matter, we will use the package XlsxWriter. First of all, we have to initialize the XlsxWriter object:


In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_dataframe.to_excel(writer,'Recommended Trades',index=False)

Now we have to create the formats we need for the .xlsx file:

* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [ ]:
background_color = '#0a0a23'
font_color = "#ffffff"


string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {   
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

Now, we must apply these formats to the columns of our .xlsx file:

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    } #types of formats

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}' #set format to this column, (e.g. A:A)
                                                   , 20 #width of column
                                                   , column_formats[column][1] #set this format
                                                  )
    writer.sheets['Recommended Trades'].write(f'{column}1' #write in this position
                                              , column_formats[column][0] #write this thing
                                              , string_format #write it in this format 
                                             )

Finally, we save the .xlsx file, which should appear in the folder in which this notebook is located:

In [ ]:
writer.save()

If you open the file, you should find a black sheet with white font called "Recommended Trades".

## References

Algorithmic Trading Using Python - Full Course : https://www.youtube.com/watch?v=xfzGZB4HhEE&t=14236s
